In [1]:
import pandas as pd
import numpy as np

In [2]:
data_path = r"/Users/loaferzk/LoaferZK/internship/soundwise/data/乳腺2020.01.01到2020.05.30.xlsx"
breastXslData = pd.read_excel(data_path, encoding = "utf-8")
breastXslData.head()

,序号,影像号,姓名,姓名拼音,性别,门诊号,住院号,检查类型,开单科室,开单医师,...,联系电话,联系地址,检查药剂,生日,阴阳性,年龄,图像质量,检查方法,是否随访,检查仪器
0,1,600437877,孙影,SunYing,女,E03618334,NaN,US,N门诊老年病,林青,...,62512388,长宁区 江苏街道 愚园路 909弄2号,NaN,19560322,阳性,64岁,乙,彩色超声,否,百胜Mylab Twice
1,2,100238987,陶燕华,TaoYanHua,女,N01994083,NaN,US,N门诊老年病,苏静英,...,13917472379,嘉定 江桥镇 曹安路 红卫村343号,NaN,19811205,阳性,38岁,乙,彩色超声,否,百胜Mylab Twice
2,3,643629,王惠英,WangHuiYing,女,H0140051X,NaN,US,N门诊老年病,胡伟国,...,13661531930,闸北区 彭浦街道 彭浦新村 84号308室,NaN,19650714,阳性,54岁,乙,彩色超声,否,百胜Mylab Twice
3,4,630684,王蕾,WangLei,女,F01135715,NaN,US,N门诊老年病,王蕾,...,62115653,静安区 江宁路街道 新闸路 1335弄18号203室,NaN,19721006,阳性,47岁,乙,"彩色超声,彩色超声",否,Philips EPIQ 7
4,5,600895897,吴思筠,WuSiJun,女,C10589549,NaN,US,N门诊老年病,林青,...,63021095,卢湾区 打浦街道 打浦路258弄5号901室,NaN,19901106,阳性,29岁,乙,彩色超声,否,百胜Mylab Twice


In [3]:
breastData = breastXslData.query('开单科室 == "乳腺肿瘤内科" | 开单科室 == "乳腺外科一" | 开单科室 == "乳腺外科二"')
breastData.head()

,序号,影像号,姓名,姓名拼音,性别,门诊号,住院号,检查类型,开单科室,开单医师,...,联系电话,联系地址,检查药剂,生日,阴阳性,年龄,图像质量,检查方法,是否随访,检查仪器
13533,13534,600098640,包丽君,BaoLiJun,女,NaN,z812981,US,乳腺外科二,高卫奇,...,13806163708,江苏省江阴市华士镇华西村华西44号,NaN,19640610,阳性,55岁,乙,乳腺中心三维超声,否,百胜MYlab60
13534,13535,300341048,蔡静,CaiJing,女,NaN,z822708,US,乳腺外科二,洪进,...,13569306166,河南省商丘市梁园区凯旋中路９６号,NaN,19790425,阳性,40岁,乙,乳腺中心三维超声,否,Mindray Re7
13535,13536,231715,蔡葵,CaiKui,女,NaN,z817092,US,乳腺外科二,朱一霏,...,18918172538,黄浦区鲁班路738弄3号1103室,NaN,19540708,阳性,65岁,乙,乳腺中心三维超声,否,百胜MYlab60
13536,13537,2429090,蔡勤,CaiQin,女,NaN,z619927,US,乳腺外科二,高卫奇,...,18919668033,安徽省合肥市经开区海馨苑2幢1604,NaN,19690518,阳性,51岁,乙,乳腺中心三维超声,否,Mindray Re7
13537,13538,300301176,曹丽骧,CaoLiXiang,女,NaN,z811032,US,乳腺外科二,高卫奇,...,15921077393,普陀区 曹杨街道 曹杨二村211号401-402室,NaN,19700305,阳性,49岁,乙,乳腺中心三维超声,否,Mindray Re7


In [4]:
save_path = r"/Users/loaferzk/LoaferZK/internship/soundwise/data/breastData_need.csv"
breastData.to_csv(save_path,encoding = "utf-8-sig")

In [1]:
import re
import pandas as pd
import numpy as np
import csv
import codecs
import os
pattern_location = re.compile(r'((?:于)[\u4e00-\u9fa5]*(?:乳).*m(?:团块))') #乳腺团块位置regex
pattern_massSize = re.compile(r'\d*\.*\d*×\d*\.*\d*×\d*\.*\d*mm')  #乳腺团块大小
pattern_shape = re.compile(r'呈不规则形|呈椭圆形|呈类圆形') #团块形状
pattern_blood = re.compile(r'CDFI示[\u4e00-\u9fa5]*血流信号')  #血流状况
pattern_echo = re.compile(r'内部呈[\u4e00-\u9fa5][\u4e00-\u9fa5]?回声') #回声(不必要)
pattern_rearEcho = re.compile(r'后方伴?回?声[\u4e00-\u9fa5]*') #后方回声
pattern_boundary = re.compile(r'边缘界限[\u4e00-\u9fa5]*') #结节边缘界限
pattern_calcification = re.compile(r'[\u4e00-\u9fa5]*钙化灶[\u4e00-\u9fa5]*') #钙化灶
pattern_category = re.compile(r'拟[^\u4e00-\u9fa5]*类') #获取拟诊断类别

In [11]:
target_path = r"/Users/loaferzk/LoaferZK/internship/soundwise/data/result.csv"
file_path = r"/Users/loaferzk/LoaferZK/internship/soundwise/data/breastData/breastData_need.csv"
fileData = pd.read_csv(file_path, encoding = "utf-8-sig")
headers = ['num','location','massSize','vh_ratio','shape','blood','echo','rearEcho','boundary','calcification','category']
# print(len(fileData["超声所见"]))
with open(target_path,"w",newline = "") as f:
    f_csv = csv.DictWriter(f,headers)
    f_csv.writeheader()
# use i to tackle diferent col data
for i in range(len(fileData["超声所见"])):
    location = []
    massSize = []
    vh_ratio = []
    shape = []
    blood = []
    echo = []
    rearEcho = []
    boundary = []
    calcification = []
    category = pattern_category.findall(fileData["超声诊断"][i].replace("\xa0","_").replace("－","-"))
    info_dict = {}
    reportList = fileData["超声所见"][i].strip(" ").split("。")
    for sentence in reportList:
        # 乳腺位置匹配
        temp_location = pattern_location.search(sentence)
        if temp_location is None:
            pass
        else:
            if "左" in temp_location.group():
                location.append("左部")
#                 info_dict["location"] = "左部"
            elif "右" in temp_location.group():
                location.append("右部")
#                 info_dict["location"] = "右部"
            else:
                pass
#             info_dict["location"] = temp_location.group()
        # 乳腺位置中团块的大小匹配
        temp_massSize = pattern_massSize.search(sentence)
        if temp_massSize is None:
            pass
        else:
            massSize.append(temp_massSize.group())
#             info_dict["massSize"] = temp_massSize.group()
            num_list = temp_massSize.group().split("×")
            try:
                vh_ratio.append(float(num_list[1])/float(num_list[0]))
            except Exception as e:
                print(num_list)
#             info_dict["vh_ratio"] = int(num_list[1])/int(num_list[0])
#             info_dict["massSize"] = pattern_massSize 在信息获取完毕后再添加至info_dict
        # 乳腺形状描述提取
        temp_shape = pattern_shape.search(sentence)
        if temp_shape is None:
            pass
        else:
            shape.append(temp_shape.group())
#             info_dict["shape"] = temp_shape.group()
        # 血流情况文本匹配
        temp_blood = pattern_blood.search(sentence)
        if temp_blood is None:
            pass
        else:
            blood.append(temp_blood.group())
#             info_dict["blood"] = temp_blood.group()
        # 回声情况匹配（回声不重要，文本中有多处描述
        temp_echo = pattern_echo.search(sentence)
        if temp_echo is None:
            pass
        else:
            echo.append(temp_echo.group())
#             info_dict["echo"] = temp_echo.group()
        # 后方回声情况匹配
        temp_rearEcho = pattern_rearEcho.search(sentence)
        if temp_rearEcho is None:
            pass
        else:
            rearEcho.append(temp_rearEcho.group())
#             info_dict["rearEcho"] = temp_rearEcho.group()
        temp_boundary = pattern_boundary.search(sentence)
        if temp_boundary is None:
            pass
        else:
            boundary.append(temp_boundary.group())
        temp_calcification = pattern_calcification.search(sentence)
        if temp_calcification is None:
            pass
        else:
            calcification.append(temp_calcification.group())
    num = len(location) # 团块数量
    if num > 1:
        info_dict["num"] = "多发"
    elif num == 1:
        info_dict["num"] = "单发"
    else:
        pass
    for i in range(num):
        try:
            info_dict["location"] = location[i]
        except Exception as e:
            info_dict["location"] = "None"
        try:
            info_dict["massSize"] = massSize[i]
        except Exception as e:
            info_dict["massSize"] = "None"
        try:
            info_dict["vh_ratio"] = vh_ratio[i]
        except Exception as e:
            info_dict["vh_ratio"] = "0"
        try:
            info_dict["shape"] = shape[i]
        except Exception as e:
            info_dict["shape"] = "None"
        try:
            info_dict["blood"] = blood[i]
        except Exception as e:
            info_dict["blood"] = "None"
        try:
            info_dict["echo"] = echo[i]
        except Exception as e:
            info_dict["echo"] = "None"
        try:
            info_dict["rearEcho"] = rearEcho[i]
        except Exception as e:
            info_dict["rearEcho"] = "None"
        try:
            info_dict["boundary"] = boundary[i]
        except Exception as e:
            info_dict["boundary"] = "None"
        try:
            info_dict["calcification"] = calcification[i]
        except Exception as e:
            info_dict["calcification"] = "None"
        try:
            info_dict["category"] = category[i]
        except Exception as e:
            info_dict["category"] = "None"
        with codecs.open(target_path,"a","gbk") as f:
            print(info_dict)
#             f.write(codecs.BOM_UTF8)
            writer = csv.DictWriter(f,headers)
#             data = list(info_dict.values())
            print("----------------")
#             print(data)
            print(type(info_dict))
            writer.writerow(info_dict)
        continue

{'num': '单发', 'location': '左部', 'massSize': '9.4×6.5×10.2mm', 'vh_ratio': 0.6914893617021276, 'shape': '呈不规则形', 'blood': 'CDFI示内边缘及中央区域见较丰富血流信号', 'echo': '内部呈低回声', 'rearEcho': '后方回声无明显改变', 'boundary': 'None', 'calcification': '未见明显钙化灶', 'category': '拟US-BI-RADS_4C类'}
----------------
<class 'dict'>
{'num': '单发', 'location': '右部', 'massSize': '18.7×14.9×14.6mm', 'vh_ratio': 0.7967914438502675, 'shape': '呈椭圆形', 'blood': 'CDFI示内未见明显血流信号', 'echo': '内部呈低回声', 'rearEcho': '后方回声无明显改变', 'boundary': '边缘界限清楚', 'calcification': '未见明显钙化灶', 'category': '拟US-BI-RADS_4C类'}
----------------
<class 'dict'>
{'num': '单发', 'location': '右部', 'massSize': '44×21×48mm', 'vh_ratio': 0.4772727272727273, 'shape': '呈不规则形', 'blood': 'CDFI示内边缘及中央区域见较丰富血流信号', 'echo': '内部呈低回声', 'rearEcho': '后方伴声衰减', 'boundary': 'None', 'calcification': 'None', 'category': '拟US-BI-RADS_5类'}
----------------
<class 'dict'>
{'num': '多发', 'location': '右部', 'massSize': '23×17×22mm', 'vh_ratio': 0.7391304347826086, 'shape': '呈不规则形', 'blood'

{'num': '单发', 'location': '右部', 'massSize': '35.0×15.9×26.6mm', 'vh_ratio': 0.4542857142857143, 'shape': '呈不规则形', 'blood': 'CDFI示内边缘区域见较丰富血流信号', 'echo': '内部呈低回声', 'rearEcho': '后方回声无明显改变', 'boundary': '边缘界限清楚', 'calcification': '未见明显钙化灶', 'category': '拟US-BI-RADS_5类'}
----------------
<class 'dict'>
{'num': '多发', 'location': '右部', 'massSize': 'None', 'vh_ratio': '0', 'shape': '呈不规则形', 'blood': 'CDFI示内边缘区域见较丰富血流信号', 'echo': '内部呈低回声', 'rearEcho': '后方伴声衰减', 'boundary': '边缘界限清楚', 'calcification': '未见明显钙化灶', 'category': '拟US-BI-RADS_5类'}
----------------
<class 'dict'>
{'num': '多发', 'location': '右部', 'massSize': 'None', 'vh_ratio': '0', 'shape': '呈不规则形', 'blood': 'CDFI示内边缘及中央区域见较丰富血流信号', 'echo': '内部呈低回声', 'rearEcho': '后方伴声衰减', 'boundary': 'None', 'calcification': '未见明显钙化灶', 'category': '拟US-BI-RADS_4C类'}
----------------
<class 'dict'>
{'num': '单发', 'location': '左部', 'massSize': 'None', 'vh_ratio': '0', 'shape': '呈不规则形', 'blood': 'None', 'echo': '内部呈低回声', 'rearEcho': '后方回声无明显改变', 'boundary':

In [66]:
import os
dropList = ["甲状腺肿","结节性甲状腺肿","亚急性甲状腺炎","亚甲炎","慢性淋巴细胞性甲状腺炎","桥本氏甲状腺炎","腺瘤","乳头状瘤","滤泡状瘤","髓样瘤","未分化癌","腺瘤样结节性甲状腺肿","肿瘤性甲状腺肿","结节性甲状腺肿","甲状腺肉芽","滤泡性腺瘤"] 
jzxFile_path = r'/Users/loaferzk/LoaferZK/internship/soundwise/data/jzx_result/jzx.xlsx'
target_path = r'/Users/loaferzk/LoaferZK/internship/soundwise/data/jzx_result'
jzxFile = pd.read_excel(jzxFile_path,encoding = 'utf-8-sig',header = 1)
jzxFile.head()

,姓名,性别,年龄,仪器型号,影像号,位置（左上，左中，左下，右上，右中，右下，峡部）,住院号,左右径,前后径,上下径,...,病理良恶性,病理报告描述,病理诊断,病理诊断.1,同侧恶性病灶数目（单发、多发、无、不可评估）,对侧恶性病灶数目（单发、多发、无、不可评估）,峡部恶性病灶数目（单发、多发、无、不可评估）,淋巴结病理报告描述,病理淋巴结转移（0无；1中央组；2中央+外侧组；3单纯外侧组）,免疫组化（该项目可不填）
0,A007王政霞f30,F,30,VolusonE8,NaN,左上,NaN,28.0,16.0,28,...,恶性,"“左侧甲状腺”乳头状癌，肿瘤最大径1.0cm,周围甲状腺组织呈桥本病伴腺瘤样增生",乳头状癌,乳头状癌,单发,无,无,（左颈部VI区）淋巴结未见癌转移（0/2）,0,NaN
1,A006韩恒生m57,m,57,迈瑞Reaona7,cs20180425338,右上,102601,12.8,8.5,6.3,...,恶性,甲状腺乳头状癌,乳头状癌,乳头状癌,单发,无,无,右侧气管食管沟组织检出淋巴结2枚，淋巴组织内均见甲状腺乳头状癌组织，另见少量甲状旁腺组织,1,NaN
2,A013陈国珍f56,F,56,迈瑞DC-7,20180205009901,左中,557194,8.2,6.2,11.4,...,恶性,（左侧）甲状腺乳头状癌，大小1.0cm×1.0cm×0.8cm，癌组织侵至甲状腺被膜下,乳头状癌,乳头状癌,单发,单发,无,送检（中央组）淋巴结未见转移（0/4）。,0,NaN
3,A019刘艳F40,F,40,飞利浦 EPIQ7,NaN,右,NaN,14.8,15.5,11.9,...,恶性,“右甲状腺”及“峡部”甲状腺乳头状癌，肿瘤大小1.3x1.0x1.2cm及直径0.2cm。累...,乳头状癌,乳头状癌,单发,无,单发,“颈右侧中央区淋巴结淋巴结”（0/3）枚未见癌转移。,0,NaN
4,A266彭金花f54,F,54,ALOKA prousoundF5,NaN,右下,NaN,9.1,8.2,8,...,恶性,“右侧甲状腺切除标本”：甲状腺微小乳头状癌（直径约0.8cm），无包膜，浸润性生长，背景为桥...,乳头状癌,乳头状癌,无,不可评估,单发,检出中央区淋巴结（4枚）未见癌转移。,0,NaN


In [9]:
# jjxjzxz = jzxFile.query('"结节性甲状腺肿" in 病理诊断').to_csv(os.path.join(target_path,"jjxjzxz.csv"), encoding = "utf-8-sig")
# yjxjzxy = jzxFile.query('"亚急性甲状腺炎" in 病理诊断|"亚甲炎" in 病理诊断').to_csv(os.path.join(target_path,"yjxjzxy.csv"),encoding = "utf-8-sig")
# mxlbxbxjzxy = jzxFile.query('"慢性淋巴细胞性甲状腺炎" in 病理诊断|"桥本氏甲状腺炎" in 病理诊断').to_csv(os.path.join(target_path,"mxlvxbxjzxy.csv"),encoding = "utf-8-sig")
# rtza = jzxFile.query('"乳头状癌" in 病理诊断').to_csv(os.path.join(target_path,"rtza.csv"), encoding = "utf-8-sig")
# lpza = jzxFile.query('"滤泡状癌" in 病理诊断').to_csv(os.path.join(target_path,"lpza.csv"), encoding = "utf-8-sig")
# sya = jzxFile.query('"髓样癌" in 病理诊断').to_csv(os.path.join(target_path,"sya.csv"), encoding = "utf-8-sig")
# wfha = jzxFile.query('"未分化癌" in 病理诊断').to_csv(os.path.join(target_path,"wfha.csv"), encoding = "utf-8-sig")
# xlyjjxjzxz = jzxFile.query('"肿瘤样结节性甲状腺肿" in 病理诊断').to_csv(os.path.join(target_path,"zlyjjxjzxz.csv"), encoding = "utf-8-sig")
# zlxjzxz = jzxFile.query('"肿瘤性甲状腺肿" in 病理诊断').to_csv(os.path.join(target_path,"zlxjzxz.csv"), encoding = "utf-8-sig")
# jjxjzxz = jzxFile.query('"结节性甲状腺肿" in 病理诊断').to_csv(os.path.join(target_path,"jjxjzxz.csv"), encoding = "utf-8-sig")
# jzxry = jzxFile.query('"甲状腺肉芽" in 病理诊断').to_csv(os.path.join(target_path,"jzxry.csv"), encoding = "utf-8-sig")
# lpxxl = jzxFile.query('"滤泡性腺瘤" in 病理诊断').to_csv(os.path.join(target_path,"lpxxl.csv"), encoding = "utf-8-sig")

In [90]:
jzxz = jzxFile.query('病理诊断=="甲状腺肿"').to_csv(os.path.join(target_path,"jzxz.csv"),encoding = "utf-8-sig")
xl = jzxFile.query('病理诊断 == "腺瘤"').to_csv(os.path.join(target_path,"xl.csv"),encoding = "utf-8-sig")
jjxjzxz = pd.DataFrame(columns=jzxFile.columns)
yjxjzxy = pd.DataFrame(columns=jzxFile.columns)
mxlbxbxjzxy = pd.DataFrame(columns=jzxFile.columns)
rtza = pd.DataFrame(columns=jzxFile.columns)
lpza = pd.DataFrame(columns=jzxFile.columns)
sya = pd.DataFrame(columns=jzxFile.columns)
wfha = pd.DataFrame(columns=jzxFile.columns)
xlyjjxjzxz = pd.DataFrame(columns=jzxFile.columns)
zlxjzxz = pd.DataFrame(columns=jzxFile.columns)
# jjxjzxz = pd.DataFrame(columns=jzxFile.columns)
jzxry = pd.DataFrame(columns=jzxFile.columns)
lpxxl = pd.DataFrame(columns=jzxFile.columns)
rest = pd.DataFrame(columns=jzxFile.columns)
# for item in jzxFile["病理诊断"]:
#     if "结节性甲状腺肿" in str(item):
# #         temp = jzxFile[jzxFile["病理诊断"]==item]
# #         jjxjzxz = jjxjzxz.append(temp)
#         pass
#     if "亚急性甲状腺炎" in str(item) or "亚甲炎" in str(item):
# #         temp = jzxFile[jzxFile["病理诊断"]==item]
# #         yjxjzxy = yjxjzxy.append(temp)
#         pass
#     if "慢性淋巴细胞性甲状腺炎" in str(item) or "桥本氏甲状腺炎" in str(item):
# #         temp = jzxFile[jzxFile["病理诊断"]==item]
# #         mxlbxbxjzxy = mxlbxbxjzxy.append(temp)
#         pass
#     if "乳头状癌" in str(item):
# #         temp = jzxFile[jzxFile["病理诊断"]==item]
# #         rtza = rtza.append(temp)
#         pass
#     if "滤泡状癌" in str(item):
# #         temp = jzxFile[jzxFile["病理诊断"]==item]
# #         lpza = lpza.append(temp)
#         pass
#     if "髓样癌" in str(item):
# #         temp = jzxFile[jzxFile["病理诊断"]==item]
# #         sya = sya.append(temp)
#         pass
#     if "未分化癌" in str(item):
# #         temp = jzxFile[jzxFile["病理诊断"]==item]
# #         wfha = wfha.append(temp)
#         pass
#     if "腺瘤样结节性甲状腺肿" in str(item):
# #         print(item)
# #         temp = jzxFile[jzxFile["病理诊断"]==item]
# #         xlyjjxjzxz = xlyjjxjzxz.append(temp)
#         pass
#     if "肿瘤性甲状腺肿" in str(item):
# #         print(item)
# #         temp = jzxFile[jzxFile["病理诊断"]==item]
# #         zlxjzxz = zlxjzxz.append(temp)
#         pass
#     if "甲状腺肉芽" in str(item):
# #         temp = jzxFile[jzxFile["病理诊断"]==item]
# #         jzxry = jzxry.append(temp)
#         pass
#     if "滤泡性腺瘤" in str(item):
# #         temp = jzxFile[jzxFile["病理诊断"]==item]
# #         lpxxl = lpxxl.append(temp)
#         pass
#     if "腺瘤" in str(item):
#         pass
#     if "甲状腺肿" in str(item):
#         pass
#     else:
# #         temp = jzxFile[jzxFile["病理诊断"]==item]
# #         rest = rest.append(temp)

In [94]:
jjxjzxz.drop_duplicates().to_csv(os.path.join(target_path,"jjxjzxz.csv"),encoding = "utf-8-sig")
yjxjzxy.drop_duplicates().to_csv(os.path.join(target_path,"yjxjzxy.csv"),encoding = "utf-8-sig")
mxlbxbxjzxy.drop_duplicates().to_csv(os.path.join(target_path,"mxlbxbxjzxy.csv"),encoding = "utf-8-sig")
rtza.drop_duplicates().to_csv(os.path.join(target_path,"rtza.csv"),encoding = "utf-8-sig")
lpza.drop_duplicates().to_csv(os.path.join(target_path,"lpza.csv"),encoding = "utf-8-sig")
sya.drop_duplicates().to_csv(os.path.join(target_path,"sya.csv"),encoding = "utf-8-sig")
wfha.drop_duplicates().to_csv(os.path.join(target_path,"wfha.csv"),encoding = "utf-8-sig")
xlyjjxjzxz.drop_duplicates().to_csv(os.path.join(target_path,"xlyjjxjzxz.csv"),encoding = "utf-8-sig")
zlxjzxz.drop_duplicates().to_csv(os.path.join(target_path,"zlxjzxz.csv"),encoding = "utf-8-sig")
# jjxjzxz = pd.DataFrame(columns=jzxFile.columns)
jzxry.drop_duplicates().to_csv(os.path.join(target_path,"jzxry.csv"),encoding = "utf-8-sig")
lpxxl.drop_duplicates().to_csv(os.path.join(target_path,"lpxxl.csv"),encoding = "utf-8-sig")
rest.drop_duplicates().to_csv(os.path.join(target_path,"rest.csv"),encoding = "utf-8-sig")

In [50]:
for item in jzxFile["病理诊断"]:
    if "腺瘤样结节性甲状腺肿" in str(item):
        print(item)
        temp = jzxFile[jzxFile["病理诊断"]==item]
        xlyjjxjzxz = xlyjjxjzxz.append(temp)

腺瘤样结节性甲状腺肿
腺瘤样结节性甲状腺肿
腺瘤样结节性甲状腺肿
腺瘤样结节性甲状腺肿
腺瘤样结节性甲状腺肿
腺瘤样结节性甲状腺肿
腺瘤样结节性甲状腺肿
腺瘤样结节性甲状腺肿
腺瘤样结节性甲状腺肿
嗜酸性腺瘤样结节性甲状腺肿
腺瘤样结节性甲状腺肿
腺瘤样结节性甲状腺肿
嗜酸性腺瘤样结节性甲状腺肿
腺瘤样结节性甲状腺肿
左甲腺瘤样结节性甲状腺肿
‎“‎左侧甲状腺‎”‎腺瘤样结节性甲状腺肿‎。
“左甲状腺”腺瘤样结节性甲状腺肿。
左腺瘤样结节性甲状腺肿‎
左”腺瘤样结节性甲状腺肿。  

“右甲状腺”腺瘤样结节性甲状腺肿
“右甲”腺瘤样结节性甲状腺肿
“左”腺瘤样结节性甲状腺肿伴囊性变。 
左甲状腺”腺瘤样结节性甲状腺肿。
腺瘤样结节性甲状腺肿多发
“右甲状腺”乳头状微癌，直径0.2cm。  “左甲状腺”腺瘤样结节性甲状腺肿。  “喉前淋巴结”3枚、“左中央组淋巴结”2枚、“右中央组淋巴结”4枚、“右侧喉返神经后方淋巴结”3枚未见肿瘤转移。


In [55]:
xlyjjxjzxz.drop_duplicates().to_csv(os.path.join(target_path,"xlyjjxjzxz.csv"),encoding = "utf-8-sig")

In [92]:
# rest = pd.DataFrame(columns=jzxFile.columns)
# mxlbxbxjzxy = pd.DataFrame(columns=jzxFile.columns)
ya = pd.DataFrame(columns=jzxFile.columns)
rutou = pd.DataFrame(columns=jzxFile.columns)
xianliu = pd.DataFrame(columns=jzxFile.columns)
for item in jzxFile["病理诊断"]:
    if "结节性甲状腺肿" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        jjxjzxz = jjxjzxz.append(temp)
#         pass
    if "亚急性甲状腺炎" in str(item) or "亚甲炎" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        yjxjzxy = yjxjzxy.append(temp)
#         pass
    if "慢性淋巴细胞性甲状腺炎" in str(item) or "桥本" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        mxlbxbxjzxy = mxlbxbxjzxy.append(temp)
#         pass
    if "乳头状癌" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        rtza = rtza.append(temp)
#         pass
    if "滤泡状癌" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        lpza = lpza.append(temp)
#         pass
    if "髓样癌" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        sya = sya.append(temp)
#         pass
    if "未分化癌" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        wfha = wfha.append(temp)
#         pass
    if "腺瘤样结节性甲状腺肿" in str(item):
#         print(item)
        temp = jzxFile[jzxFile["病理诊断"]==item]
        xlyjjxjzxz = xlyjjxjzxz.append(temp)
#         pass
    if "肿瘤性甲状腺肿" in str(item):
#         print(item)
        temp = jzxFile[jzxFile["病理诊断"]==item]
        zlxjzxz = zlxjzxz.append(temp)
#         pass
    if "甲状腺肉芽" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        jzxry = jzxry.append(temp)
#         pass
    if "滤泡性腺瘤" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        lpxxl = lpxxl.append(temp)
#         pass
    if "亚" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
#         print(temp)
        ya = ya.append(temp)
#         pass
    if "乳头状" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        rutou = rutou.append(temp)
#         pass
    if "腺瘤" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
#         print("腺瘤")
        xianliu = xianliu.append(temp)
    else:
#         pass
        temp = jzxFile[jzxFile["病理诊断"]==item]
#         print(temp["病理诊断"])
        rest = rest.append(temp)

In [95]:
# rest.drop_duplicates().to_csv(os.path.join(target_path,"rest.csv"),encoding = "utf-8-sig")
# mxlbxbxjzxy.drop_duplicates().to_csv(os.path.join(target_path,"mxlbxbxjzxy.csv"),encoding = "utf-8")
ya.drop_duplicates().to_csv(os.path.join(target_path,"ya.csv"),encoding = "utf-8-sig")
# rutou.drop_duplicates().to_csv(os.path.join(target_path,"rutou.csv"),encoding = "utf-8-sig")
xianliu.drop_duplicates().to_csv(os.path.join(target_path,"xianliu.csv"),encoding = "utf-8-sig")

In [82]:
len(ya)

2

In [93]:
qiaoben = pd.DataFrame(columns = jzxFile.columns)
zhong = pd.DataFrame(columns = jzxFile.columns)
ruTou_ = pd.DataFrame(columns = jzxFile.columns)
lvpao = pd.DataFrame(columns = jzxFile.columns)
weifenhua = pd.DataFrame(columns = jzxFile.columns)
suiyang = pd.DataFrame(columns = jzxFile.columns)
linbaliu = pd.DataFrame(columns = jzxFile.columns)
qita = pd.DataFrame(columns = jzxFile.columns)
for item in jzxFile["病理诊断"]:
    if "桥本" in str(item) or "乔本" in str(item) or "慢性" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        qiaoben = qiaoben.append(temp)
    if "肿" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        zhong = zhong.append(temp)
    if "乳头" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        ruTou_ = ruTou_.append(temp)
    if "滤泡" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        lvpao = lvpao.append(temp)
    if "未分化" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        weifenhua = weifenhua.append(temp)
    if "髓样" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        suiyang = suiyang.append(temp)
    if "淋巴瘤" in str(item):
        temp = jzxFile[jzxFile["病理诊断"]==item]
        linbaliu = linbaliu.append(temp)
    else:
        temp = jzxFile[jzxFile["病理诊断"]==item]
        qita = qita.append(temp)

In [89]:
len(xianliu)

0

In [97]:
qiaoben.drop_duplicates().to_csv(os.path.join(target_path,"v2","qiaoben.csv"),encoding = "utf-8-sig")
zhong.drop_duplicates().to_csv(os.path.join(target_path,"v2","zhong.csv"),encoding = "utf-8-sig")
ruTou_.drop_duplicates().to_csv(os.path.join(target_path,"v2","ruTou_.csv"),encoding = "utf-8-sig")
lvpao.drop_duplicates().to_csv(os.path.join(target_path,"v2","lvpao.csv"),encoding = "utf-8-sig")
weifenhua.drop_duplicates().to_csv(os.path.join(target_path,"v2","weifenhua.csv"),encoding = "utf-8-sig")
suiyang.drop_duplicates().to_csv(os.path.join(target_path,"v2","suiyang.csv"),encoding = "utf-8-sig")
linbaliu.drop_duplicates().to_csv(os.path.join(target_path,"v2","linbaliu.csv"),encoding = "utf-8-sig")
qita.drop_duplicates().to_csv(os.path.join(target_path,"v2","qita.csv"),encoding = "utf-8-sig")

In [120]:
for item in qita["病理诊断"]:
#     if "桥本" in str(item) or "乔本" in str(item) or "慢性" in str(item):
#         qita.drop(qita[qita["病理诊断"]==item].index,inplace = True)
    if "肿" in str(item):
        qita.drop(qita[qita["病理诊断"]==item].index,inplace = True)
    if "乳头" in str(item):
        qita.drop(qita[qita["病理诊断"]==item].index,inplace = True)
#         print(len(qita))
    if "滤泡" in str(item):
        qita.drop(qita[qita["病理诊断"]==item].index,inplace = True)
    if "未分化" in str(item):
        qita.drop(qita[qita["病理诊断"]==item].index,inplace = True)
    if "髓样" in str(item):
        qita.drop(qita[qita["病理诊断"]==item].index,inplace = True)
    if "淋巴瘤" in str(item):
        qita.drop(qita[qita["病理诊断"]==item].index,inplace = True)

In [122]:
for item in qita["病理诊断"]:
    if "亚" in str(item):
        qita.drop(qita[qita["病理诊断"]==item].index,inplace = True)
    if "腺瘤" in str(item):
        qita.drop(qita[qita["病理诊断"]==item].index,inplace = True)
print(len(qita.drop_duplicates()))
qita.drop_duplicates().to_csv(os.path.join(target_path,"v2","qita.csv"),encoding = "utf-8-sig")

62


In [113]:
len(jzxFile)

3708

In [3]:
zhongData = pd.read_excel(r"/Users/loaferzk/LoaferZK/internship/soundwise/data/zhong.xlsx",encoding = "utf-8-sig")
zhongData.head()

,Unnamed: 0,姓名,性别,年龄,仪器型号,影像号,位置（左上，左中，左下，右上，右中，右下，峡部）,住院号,左右径,前后径,...,病理良恶性,病理报告描述,病理诊断,病理诊断.1,同侧恶性病灶数目（单发、多发、无、不可评估）,对侧恶性病灶数目（单发、多发、无、不可评估）,峡部恶性病灶数目（单发、多发、无、不可评估）,淋巴结病理报告描述,病理淋巴结转移（0无；1中央组；2中央+外侧组；3单纯外侧组）,免疫组化（该项目可不填）
0,42,A138魏俊菊f54,F,54,西门子S2000,US18053806,右中,18020380,13.3,13.8,...,恶性,甲状腺结节性甲状腺肿伴钙化及骨化，钙化组织内查见乳头状癌。,甲状腺结节性甲状腺肿伴钙化及骨化，钙化组织内查见乳头状癌。,甲状腺肿伴钙化及骨化，钙化组织内查见乳头状癌。,单发,无,无,NaN,0,NaN
1,122,A001陆彧M33,M,33,NaN,100354593,左下,z337320,NaN,11.9,...,良性,“左”甲状腺肉芽肿性炎,甲状腺肉芽肿性炎,甲状腺肉芽肿性炎,无,不可评估,不可评估,NaN,0,NaN
2,139,A001刘红F42,F,42,NaN,NaN,NaN,Z352494,NaN,19.0,...,良性,“左”腺瘤样甲状腺肿伴嗜酸性变,腺瘤样甲状腺肿伴嗜酸性变,腺瘤样甲状腺肿伴嗜酸性变,无,无,无,NaN,0,NaN
3,141,A001徐丽娟F49,F,49,NaN,NaN,NaN,Z352878,NaN,16.0,...,良性,结节性甲状腺肿伴腺瘤样结节形成,结节性甲状腺肿伴腺瘤样结节形成,结节性甲状腺肿伴腺瘤样结节形成,无,不可评估,不可评估,NaN,0,NaN
4,667,A166喻启英f62,f,62,GE Logiq S7,NaN,左中,1191949,18.6,12.3,...,良性,结节性甲状腺肿伴腺瘤样结节形成,结节性甲状腺肿伴腺瘤样结节形成,结节性甲状腺肿伴腺瘤样结节形成,无,无,无,无,0,NaN


In [4]:
jjxjzxzbxlyjj = pd.DataFrame(columns=zhongData.columns)
xlyjjxjzxz = pd.DataFrame(columns=zhongData.columns)
xlyjzxz = pd.DataFrame(columns=zhongData.columns)
jjxjzxzbxlyzs = pd.DataFrame(columns=zhongData.columns)
jjxjzxzbxlxc = pd.DataFrame(columns=zhongData.columns)
jzxjjxjzxz = pd.DataFrame(columns=zhongData.columns)
jjxjzxzbxwh = pd.DataFrame(columns=zhongData.columns)
jjxjzxzblb = pd.DataFrame(columns=zhongData.columns)
nxb = pd.DataFrame(columns=zhongData.columns)
cx = pd.DataFrame(columns=zhongData.columns)

In [5]:
for item in zhongData["病理诊断"]:
    if "结节性甲状腺肿伴腺瘤样结节" in str(item):
        temp = zhongData[zhongData["病理诊断"]==item]
        jjxjzxzbxlyjj = jjxjzxzbxlyjj.append(temp)
    if "腺瘤样结节性甲状腺肿" in str(item):
        temp = zhongData[zhongData["病理诊断"]==item]
        xlyjjxjzxz = xlyjjxjzxz.append(temp)
    if "腺瘤样甲状腺肿" in str(item):
        temp = zhongData[zhongData["病理诊断"]==item]
        xlyjzxz = xlyjzxz.append(temp)
    if "结节性甲状腺肿伴腺瘤样增生" in str(item):
        temp = zhongData[zhongData["病理诊断"]==item]
        jjxjzxzbxlyzs = jjxjzxzbxlyzs.append(temp)
    if "结节性甲状腺肿伴腺瘤形成" in str(item):
        temp = zhongData[zhongData["病理诊断"]==item]
        jjxjzxzbxlxc = jjxjzxzbxlxc.append(temp)
    if "甲状腺结节性甲状腺肿" in str(item):
        temp = zhongData[zhongData["病理诊断"]==item]
        jzxjjxjzxz = jzxjjxjzxz.append(temp)
    if "结节性甲状腺肿伴纤维化" in str(item):
        temp = zhongData[zhongData["病理诊断"]==item]
        jjxjzxzbxwh = jjxjzxzbxwh.append(temp)
    if "结节性甲状腺肿伴淋巴" in str(item):
        temp = zhongData[zhongData["病理诊断"]==item]
        jjxjzxzblb = jjxjzxzblb.append(temp)
    if "囊性变" in str(item):
        temp = zhongData[zhongData["病理诊断"]==item]
        nxb = nxb.append(temp)
    if "出血" in str(item):
        temp = zhongData[zhongData["病理诊断"]==item]
        cx = cx.append(temp)

In [7]:
zhongTargetPath = r"/Users/loaferzk/LoaferZK/internship/soundwise/data/zhong"
jjxjzxzbxlyjj.drop_duplicates().to_csv(os.path.join(zhongTargetPath,"jjxjzxzbxlyjj.csv"),encoding = "utf-8-sig")
xlyjjxjzxz.drop_duplicates().to_csv(os.path.join(zhongTargetPath,"xlyjjxjzxz.csv"),encoding = "utf-8-sig")
xlyjzxz.drop_duplicates().to_csv(os.path.join(zhongTargetPath,"xlyjzxz.csv"),encoding = "utf-8-sig")
jjxjzxzbxlyzs.drop_duplicates().to_csv(os.path.join(zhongTargetPath,"jjxjzxzbxlyzs.csv"),encoding = "utf-8-sig")
jjxjzxzbxlxc.drop_duplicates().to_csv(os.path.join(zhongTargetPath,"jjxjzxzbxlxc.csv"),encoding = "utf-8-sig")
jzxjjxjzxz.drop_duplicates().to_csv(os.path.join(zhongTargetPath,"jzxjjxjzxz.csv"),encoding = "utf-8-sig")
jjxjzxzbxwh.drop_duplicates().to_csv(os.path.join(zhongTargetPath,"jjxjzxzbxwh.csv"),encoding = "utf-8-sig")
jjxjzxzblb.drop_duplicates().to_csv(os.path.join(zhongTargetPath,"jjxjzxzblb.csv"),encoding = "utf-8-sig")
nxb.drop_duplicates().to_csv(os.path.join(zhongTargetPath,"nxb.csv"),encoding = "utf-8-sig")
cx.drop_duplicates().to_csv(os.path.join(zhongTargetPath,"cx.csv"),encoding = "utf-8-sig")